## 使用LangGraph构建代码助手  

+ 针对特定问题参考材料迭代式的生成代码
+ 从用户指定的一组文档开始
+ 使用长上下文LLM来提取它并执行RAG来回答基于它的问题
+ 调用一个工具来生成结构化输出
+ 将解决方案返回给用户之前，将执行两个单元测试（检查导入和代码执行）

In [1]:
# 安装依赖
!pip install -U langchain_community langchain-openai langchain-deepseek langgraph bs4 langchain-anthropic

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.4 MB/s  0:00:01 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 928.3 kB/s  0:00:01 eta 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.107.1
    Uninstalling openai-1.107.1:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/8 [openai]
      Successfully uninstalled openai-1.107.1━━━━━━━━━━━━━━━━━ 2/8 [openai]
  Attempting uninstall: anthropic━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/8 [openai]
    Found existing installation: anthropic 0.64.0━━━━━━━━━━━━━ 2/8 [openai]
    Uninstalling anthropic-0.64.0:━━━━━━━━━━━━━━━━━━━━━━━━ 3/8 [anthropic]
      Successfully uninstalled anthropic-0.64.0━━━━━━━━━━━━━━━ 3/8 [anthropic]
  Attempting uninstall: langchain-openai0m━━━━━━━━━━━━━━━━━━━━━━━━ 3/8 [anthropic]
    Found existing installation: langchain-openai 0.3.33━━━━━━ 3/8 [anthropic]
    Uninstalling langchain-openai-0.3.33:━━━━━━━━━━━━━━━━━━━━━ 3/

### 加载LCEL文档

In [2]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL 文档
url = "https://python.langchain.com/docs/concepts/lcel/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [3]:
concatenated_content

'LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this page Create an agent Core benefitsLangChain overviewCopy pageLangChain is an open source framework with a pre-built agent architecture and integrations for any model or tool — so you can build agents that adapt as fast as the ecosystem evolvesCopy pageLangChain is the easiest way to start building agents and a

### 单智能体实现

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv

load_dotenv(".env", override=True)

llm = ChatDeepSeek(
    model=os.environ.get("DEEPSEEK_MODEL"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    temperature=0.0,
)

code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """你是一位精通LCEL(LangChain表达式语言)的编程助手。
            这里是LCEL文档的完整集合：\n ------ \n {context} \n ------ \n
            请根据上述提供的文档回答用户问题。确保你提供的任何代码都可以执行，
            包含所有必要的导入和已定义的变量。请按照以下结构组织你的回答：首先描述代码解决方案，
            然后列出导入语句，最后给出功能完整的代码块。以下是用户问题：
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)


# 数据模型
class Code(BaseModel):
    """LCEL问题代码解决方案的模式"""

    prefix: str = Field(description="问题和解决方案的描述")
    imports: str = Field(description="代码块导入语句")
    code: str = Field(description="不包括导入语句的代码块")


code_gen_chain_oai = code_gen_prompt | llm.with_structured_output(Code)
question = "如何在LCEL中构建链?"

solution = code_gen_chain_oai.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)

solution

Code(prefix='在LCEL中构建链的核心是使用管道操作符 `|` 将组件连接起来。LCEL（LangChain表达式语言）允许您通过简单的管道语法创建复杂的链。以下是一个完整的示例，展示如何构建一个基本的LCEL链，包括模型调用、提示模板和输出解析器。', imports='from langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_openai import ChatOpenAI', code='# 1. 创建模型\nmodel = ChatOpenAI(model="gpt-3.5-turbo")\n\n# 2. 创建提示模板\nprompt = ChatPromptTemplate.from_messages([\n    ("system", "你是一个有用的助手"),\n    ("user", "{input}")\n])\n\n# 3. 创建输出解析器\noutput_parser = StrOutputParser()\n\n# 4. 使用管道操作符构建链\nchain = prompt | model | output_parser\n\n# 5. 调用链\nresult = chain.invoke({"input": "什么是LCEL？"})\nprint(result)\n\n# 6. 流式输出示例\nprint("\\n流式输出：")\nfor chunk in chain.stream({"input": "什么是LCEL？"}):\n    print(chunk, end="", flush=True)')

In [20]:
import os
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv(".env", override=True)

llm = ChatDeepSeek(
    model=os.environ.get("DEEPSEEK_MODEL"),
    base_url=os.environ.get("DEEPSEEK_API_BASE"),
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    temperature=0.0,
)

# 强制使用工具的提示
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """<instructions>
            你是一位精通LCEL(LangChain表达式语言)的编程助手。
            这里是LCEL文档的完整集合：\n ------ \n {context} \n ------ \n
            请根据上述提供的文档回答用户问题。确保你提供的任何代码都可以执行，
            包含所有必要的导入和已定义的变量。请按照以下结构组织你的回答：1)描述代码解决方案的前言，
            2)导入语句，3)功能完整的代码块。调用code工具来正确构建输出。
            </instructions>
            以下是用户问题：
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

# 定义结构化输出
# include_raw=True，返回值不仅包含“解析后的结果”，还包含“原始消息（raw）”和“解析错误（parsing_error）”
structured_llm_claude = llm.with_structured_output(Code, include_raw=True)


# 可选：检查工具使用是否出现错误
def check_claude_output(tool_output):
    """检查解析错误或未能调用工具"""

    # 解析错误
    if tool_output["parsing_error"]:
        # 报告输出和解析错误
        print("解析错误！")
        raw_output = str(tool_output["raw"].content)
        error = tool_output["parsing_error"]
        raise ValueError(
            f"解析输出时出错！请确保调用工具。输出：{raw_output}。\n 解析错误：{error}"
        )
    # 未调用工具
    elif not tool_output["parsed"]:
        # 报告未调用工具
        print("未调用工具！")
        raise ValueError("你没用使用提供的工具！请确保调用工具来构建输出。")
    return tool_output


# 带输出检查的链
code_chain_claude_raw = code_gen_prompt | structured_llm_claude | check_claude_output


def insert_errors(inputs):
    """在消息中插入工具解析错误"""

    # 获取错误
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        (
            "assistant",
            f"重试。你需要修复解析错误：{error}。你必须调用提供的工具。",
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }


fallback_chain = insert_errors | code_chain_claude_raw
N = 3  # 最大重试次数
# 自动化重试机制
# 如果 code_chain_claude_raw 抛出异常，程序不会直接崩溃，而是转而执行 fallbacks 列表里的链
code_gen_chain_retry = code_chain_claude_raw.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)


def parse_output(solution):
    """当我们在结构化输出中添加 include_raw=True时，
    它将返回一个包含'raw'、'parsed'、'parsing_error'的字典。"""

    return solution["parsed"]


# 可选：带重试功能，用于纠正未能调用工具的情况
code_gen_chain = code_gen_chain_retry | parse_output

# 不重试
# code_gen_chain = code_gen_prompt | structured_llm_claude | parse_output

In [21]:
question = "如何在LCEL中构建链?"

solution = code_gen_chain.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)

solution

Code(prefix='在LCEL中构建链的核心是使用管道操作符 `|` 将不同的组件连接起来。LCEL（LangChain表达式语言）提供了一种声明式的方式来构建链，使得代码更加简洁和可读。以下是一个完整的示例，展示如何使用LCEL构建一个简单的问答链。', imports='from langchain_openai import ChatOpenAI\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.output_parsers import StrOutputParser', code='# 1. 创建模型\nmodel = ChatOpenAI(model="gpt-3.5-turbo")\n\n# 2. 创建提示模板\nprompt = ChatPromptTemplate.from_messages([\n    ("system", "你是一个有用的助手。"),\n    ("user", "{input}")\n])\n\n# 3. 创建输出解析器\noutput_parser = StrOutputParser()\n\n# 4. 使用LCEL构建链\nchain = prompt | model | output_parser\n\n# 5. 运行链\nresult = chain.invoke({"input": "什么是LCEL？"})\nprint(result)\n\n# 6. 流式输出示例\nprint("\\n流式输出示例：")\nfor chunk in chain.stream({"input": "什么是LCEL？"}):\n    print(chunk, end="", flush=True)')

### 多智能体实现

In [22]:
from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    表示我们图的状态

    Attributes:
        error: 用户控制流的二进制标志，表示是否触发了测试错误
        messages: 包含用户问题、错误消息、推理过程
        generation: 代码解决方案
        iterations: 尝试次数
    """

    error: str
    messages: list
    generation: str
    iterations: int

In [23]:
# 创建节点

# 最大尝试次数
max_iterations = 3

# 是否开启反思
# flag = 'reflect'
flag = "do not reflect"


# 节点
def generate(state: GraphState):
    """
    生成代码解决方案

    参数：
        state(dict): 图状态

    返回：
        state(dict): 向状态添加新键，generation
    """
    print("---正在生成代码解决方案---")
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # 我们因错误被重新路由到生成
    if error == "yes":
        messages += [
            (
                "user",
                "现在，请重试。调用code工具来构建包含前言、导入和代码块的输出：",
            )
        ]

    # 生成内容
    code_solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )

    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n 导入: {code_solution.imports} \n 代码: {code_solution.code}",
        )
    ]

    # 增加计数
    iterations += 1
    return {
        "messages": messages,
        "generation": code_solution,
        "iterations": iterations,
    }


def code_check(state: GraphState):
    """
    检查代码

    参数：
        state(dict): 当前图状态

    返回：
        state(dict): 向状态添加新键，error
    """
    print("---正在检查代码---")
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # 获取解决方案组件
    imports = code_solution.imports
    code = code_solution.code

    # 检查导入
    try:
        exec(imports)
    except Exception as e:
        print("---代码导入检查：失败---")
        error_message = [
            (
                "user",
                f"你的解决方案未通过导入测试：{e}",
            )
        ]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "error": "yes",
            "iterations": iterations,
        }

    # 检查执行
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---代码块检查：失败---")
        error_message = [
            (
                "user",
                f"你的解决方案未通过代码执行测试：{e}",
            )
        ]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "error": "yes",
            "iterations": iterations,
        }

    # 无错误
    print("---代码检查：成功---")
    return {
        "generation": code_solution,
        "messages": messages,
        "error": "no",
        "iterations": iterations,
    }


def reflect(state: GraphState):
    """
    对错误进行反思

    参数：
        state(dict): 当前图状态

    返回：
        state(dict): 向状态添加新键，generation
    """
    print("---正在生成代码解决方案---")

    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # 提示反思

    # 添加反思
    reflections = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [
        (
            "assistant",
            f"以下是对错误的放肆 {reflections}",
        )
    ]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


# 定义条件边
def decide_to_finish(state: GraphState):
    """
    检查是否结束

    参数：
        state(dict): 当前图状态

    返回：
        str: 要调用的下一个节点
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---决定：完成---")
        return "end"
    else:
        print("---决定：重新尝试解决方案---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"

In [25]:
# 创建工作流
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(GraphState)

# 添加节点
workflow.add_node("generate", generate)
workflow.add_node("check_code", code_check)
workflow.add_node("reflect", reflect)

# 构建图
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "generate")


app = workflow.compile()

In [26]:
# Test
# question = "如何在runnable中传递原始的输入?"
question = "如何并行执行两条链?"
solution = app.invoke(
    {
        "messages": [("user", question)],
        "iterations": 0,
        "error": "",
    }
)

solution

---正在生成代码解决方案---
---正在检查代码---
=== 同步执行 ===
---代码块检查：失败---
---决定：重新尝试解决方案---
---正在生成代码解决方案---
---正在检查代码---
=== 同步执行 ===
---代码块检查：失败---
---决定：重新尝试解决方案---
---正在生成代码解决方案---
---正在检查代码---
=== 同步执行 ===
---代码块检查：失败---
---决定：完成---


{'error': 'yes',
 'messages': [('user', '如何并行执行两条链?'),
  ('assistant',
   '在LangChain中并行执行两条链可以通过多种方式实现。根据提供的文档，LangChain提供了标准化的模型接口和灵活的代理架构。以下是几种并行执行链的方法：\n\n1. 使用`RunnableParallel`：这是LCEL中专门用于并行执行多个可运行对象的组件\n2. 使用`RunnableLambda`结合异步执行：通过异步编程实现并行\n3. 使用`RunnableBranch`：根据条件并行执行不同的分支\n\n这里我将展示使用`RunnableParallel`的方法，这是最直接和推荐的方式。 \n 导入: from langchain_core.runnables import RunnableParallel, RunnableLambda\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_openai import ChatOpenAI\nimport asyncio\nfrom typing import Dict, Any \n 代码: # 创建两个简单的链作为示例\n# 第一个链：获取天气信息\ndef get_weather(city: str) -> str:\n    """获取城市天气信息"""\n    return f"It\'s always sunny in {city}!"\n\n# 第二个链：格式化问候语\ndef format_greeting(name: str) -> str:\n    """格式化问候语"""\n    return f"Hello, {name}! How can I help you today?"\n\n# 创建并行执行的链\nparallel_chain = RunnableParallel(\n    weather=RunnableLambda(lambda x: get_weather(x.get("city", "unknown"))),\n    greeting=RunnableLambda(lambda x: format_greeti